# Simplicial morphisms

The software ``Kenzo`` implements simplicial morphisms in a way analogous to chain complex morphisms.

## Representation of a  simplicial morphism

A simplicial morphism is  an instance of the class ``SIMPLICIAL-MRPH``, subclass of the class ``MORPHISM``.

```lisp
(DEFCLASS SIMPLICIAL-MRPH (morphism)
  ((sintr :type sintr :initarg :sintr :reader sintr)))
```

It has one slot of its own:
> ``sintr``, an object of type ``SINTR``, in fact a lisp function defining the morphism between the source and target simplicial sets. It must have  2 parameters:  a dimension (an integer) and a geometric simplex of this dimension (a generator of any type) . It must return  an abstract simplex, image in the target simplicial set of this geometric simplex.

A printing method has been associated to the class ``SIMPLICIAL-MRPH``. A string like ``[K``*n*  ``Simplicial-Morphism]`` is the  external representation of an instance of such a class,  where *n* is the number plate of the ``Kenzo`` object.

## The function ``build-smmr``

To facilitate the construction of instances of the class ``SIMPLICIAL-MRPH`` and to free  the user to call the standard constructor ``make-instance``, the software provides the function

> ``build-smmr :sorc`` *sorc* ``:trgt`` *trgt* ``:degr`` *degr* ``:sintr`` *sintr* ``:intr`` *intr* ``:strt`` *strt* ``:orgn`` *orgn*

defined with keyword parameters and returning an instance of the class ``SIMPLICIAL-MRPH``. The keyword arguments of ``build-smmr`` are:

- *sorc*, the source object, an object of type ``SIMPLICIAL-SET``.
- *trgt*, the target object, an object of type ``SIMPLICIAL-SET``.
- *degr*, the degree of the morphism, an integer. In this chapter, we  consider only the 0 degree case (the usual one). The case -1 is particularly important: it allows to implement the notion of twisting operator defining a fibration (See the chapter Fibration).
- *sintr*, the internal lisp function defining the effective mapping between simplicial sets. If the integer *degr* is 0 and if the following keyword argument *intr* is omitted, then the function ``build-smmr`` builds a lisp function implementing the induced mapping between the underlying source and target chain complexes. This function is installed in the slot ``intr``. The strategy is then set to ``:gnrt``.
- ``intr``, a user defined morphism for the underlying chain complexes. This argument is optional and taken in account only if the degree is 0 and in this case, supersedes the previous derived  mapping. The strategy is then mandatory. If the degree is not null, the implementor has decided  to set the corresponding slot to ``NIL``.
- *strt*, the strategy, i.e. ``:gnrt`` or ``:cmbn`` attached to the previous function.
- *orgn*, a relevant comment list.

After a call to ``build-smmr``, the simplicial morphism instance is added to a list of previously constructed  ones (``*smmr-list*``). As the other similar lists, the list ``*smmr-list*`` may be cleared by the function ``cat-init``.

The effective application of a simplicial morphism upon arguments, is realized with the macro ``?`` which  calls the adequate method defined for this kind of objects.

After a call to ``build-smmr``, the simplicial morphism instance is added to a list of previously constructed  ones (``*smmr-list*``). As the other similar lists, the list ``*smmr-list*`` may be cleared by the function ``cat-init``.

The effective application of a simplicial morphism upon arguments, is realized with the macro ``?`` which  calls the adequate method defined for this kind of objects.

``? &rest`` *args* *[Macro]*
> Versatile macro for applying a simplicial morphism indifferently as ``(?`` *smmr dmns absm-or-gmsm*``)`` or  ``(?`` *smmr cmbn*``)``. In the first case, the third argument is either an abstract simplex or a geometric one. In the second case, if the second argument is a combination, *smmr* is then considered as a chain complex  morphism and it is the function in the slot ``intr`` which is applied, so that it makes sense only if the degree of the mapping is 0.

### Examples

In the following examples, we work with $\Delta_3$. We define three simplicial morphisms, ``sm1``, ``sm2`` and ``sm3``. In ``sm1``, the mapping is the identity mapping and we can see that this identity mapping has been propagated on the underlying chain complex.

In [1]:
(def d3 (delta 3))

[K1 Simplicial-Set]

In [2]:
(def sm1 (build-smmr
            :sorc d3 :trgt d3 :degr 0
            :sintr #'(lambda (dmns gmsm)
                             (declare (ignore dmns))
                             (absm 0 gmsm))
            :orgn '(identity delta-3)))

SIMPLE-STYLE-WARNING: 
  #<SB-INT:SIMPLE-STYLE-WARNING "undefined ~(~A~): ~S" {10053D8EA3}>


[K6 Simplicial-Morphism K1 -> K1]

In [3]:
(? sm1 2 7)

<AbSm - 7>

In [4]:
(? sm1 1 (absm 1 1))

<AbSm 0 1>

In [5]:
(? sm1 (cmbn 2 1 7))


----------------------------------------------------------------------{CMBN 2}
<1 * 7>
------------------------------------------------------------------------------


In the simplicial morphism ``sm2``, the mapping is the "null" mapping. In fact, it consists in  applying any abstract simplex in dimension $n$ upon the $n$-degenerate base point. Of course, in terms of chain complex, the corresponding mapping applies any chain complex element upon the null combination of same degree.

In [6]:
(def sm2 (build-smmr
            :sorc d3 :trgt d3 :degr 0
            :sintr #'(lambda (dmns gmsm)
                             (declare (ignore gmsm))
                             (absm (mask dmns) 1))    ;;; mask(n)=2^n - 1 
            :orgn '(null delta-3)))

SIMPLE-STYLE-WARNING: 
  #<SB-INT:SIMPLE-STYLE-WARNING "undefined ~(~A~): ~S" {1005794753}>


[K7 Simplicial-Morphism K1 -> K1]

In [7]:
(? sm2 0 4)

<AbSm - 1>

In [8]:
(? sm2 2 7)

<AbSm 1-0 1>

In [9]:
(? sm2 1 (absm 1 1))

<AbSm 0 1>

In [10]:
(? sm2 3 15)

<AbSm 2-1-0 1>

In [11]:
(? sm2 (cmbn 3 2 15))


----------------------------------------------------------------------{CMBN 3}
------------------------------------------------------------------------------


In the simplicial morphism ``sm3``, we keep the same "null" mapping as in ``sm2`` but we choose as mapping for the underlying chain complex the opposite of any chain complex element.

In [12]:
(def sm3 (build-smmr
            :sorc d3 :trgt d3 :degr 0
            :sintr #'(lambda (dmns gmsm)
                             (declare (ignore gmsm))
                             (absm (mask dmns) 1))
            :intr #'cmbn-opps
            :strt :cmbn
            :orgn '(null and opposite delta-3)))

SIMPLE-STYLE-WARNING: 
  #<SB-INT:SIMPLE-STYLE-WARNING "undefined ~(~A~): ~S" {1005A398C3}>


[K8 Simplicial-Morphism K1 -> K1]

In [13]:
(? sm3 3 15)

<AbSm 2-1-0 1>

In [14]:
(? sm3 (cmbn 3 7 15))


----------------------------------------------------------------------{CMBN 3}
<-7 * 15>
------------------------------------------------------------------------------


In [15]:
(? sm3 (cmbn 3 11 (dlop-ext-int '(0 1 2 3))))


----------------------------------------------------------------------{CMBN 3}
<-11 * 15>
------------------------------------------------------------------------------


### Lisp files concerned in this chapter

``simplicial-mrphs.lisp``.

[``classes.lisp``, ``macros.lisp``, ``various.lisp``].